## Downloading the data

The dataset is retrieved from the website http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz using as a reference the following research paper


> Lukas Bossard, Matthieu Guillaumin, Luc Van Gool - Food-101 – Mining Discriminative Components with Random Forests

The Food-101 data set consists of images from Foodspotting [1]. Any use beyond
   scientific fair use must be negociated with the respective picture owners
   according to the Foodspotting terms of use [2].

[1] http://www.foodspotting.com/
[2] http://www.foodspotting.com/terms/

In [1]:
#importing essential modules
import pandas as pd
import numpy as np
import os
from os import path
import time
from random import seed, choice
import shutil

In [5]:
#deleting the data folder in case a cleaning is required
#shutil.rmtree("../data")

In [ ]:
#the data will be downloaded and automatically extracted to the data folder ../data/food-101/images
%mkdir ../data
!wget -O ../data/food-101.tar.gz http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -zxf ../data/food-101.tar.gz -C ../data

## Organise train and test set

In [2]:
#dividing into train and test set using the json metadata 

metafolder = "../data/food-101/meta/"
train_meta = pd.read_json(path_or_buf = metafolder + "train.json")
test_meta = pd.read_json(path_or_buf = metafolder + "test.json")

> ### Organising metdatada for training, testing and validation

In [3]:
#organising metadata for training, testing and validation
validation_split = 0.2
val_split_idx = int(np.floor(train_meta.shape[0]*validation_split))

#folder with all the food images
data_dir = "../data/food-101/images/"
folders_sorted = sorted(os.listdir(data_dir))

#number of categories to randomly select
nc = 5

#selecting a randomn subset of categories
seed(42)

selection = []
while len(selection) < nc:
    pick = choice(folders_sorted)
    if pick not in set(selection):
        selection.append(pick)
        
print("Selected categories : {}".format(', '.join(map(str, selection))))

Selected categories : ramen, carrot_cake, beef_carpaccio, strawberry_shortcake, escargots


In [4]:
#create folder with data to upload to s3
%mkdir ../data/s3_train_data
%mkdir ../data/s3_train_data/train_img 
%mkdir ../data/s3_train_data/valid_img
%mkdir ../data/test_img

train_meta = train_meta[selection].iloc[:train_meta.shape[0] - val_split_idx]

valid_meta = train_meta[selection].iloc[train_meta.shape[0] - val_split_idx:]

test_meta = test_meta[selection]

#Setting train, validation and test set target folder
#target folder - train
trainfolder = "../data/s3_train_data/train_img/"

#target folder - validation
validfolder = "../data/s3_train_data/valid_img/"

#target folder -test
testfolder = "../data/test_img/"

print("{} images used for training".format(train_meta.shape[0]*train_meta.shape[1]))
print("{} images used for validation".format(valid_meta.shape[0]*valid_meta.shape[1]))
print("{} images used for testing".format(test_meta.shape[0]*test_meta.shape[1]))

mkdir: cannot create directory ‘../data/s3_train_data’: File exists
mkdir: cannot create directory ‘../data/s3_train_data/train_img’: File exists
mkdir: cannot create directory ‘../data/s3_train_data/valid_img’: File exists
mkdir: cannot create directory ‘../data/test_img’: File exists
3000 images used for training
750 images used for validation
1250 images used for testing


In [7]:
#dividing into train and test set using the json metadata 

def organise_files_from_df(df, datafolder, datatarget):
    """
    This function moves files contained in a folder (datafolder) to a target path (datatarget),
    based on the information contained on a dataframe (df) where each column corresponds to a 
    class name (sub-folder). Every column of the dataset contains a list of filenames to be moved.
    """
    
    #creating target folder
    if not path.exists(datatarget):
        os.mkdir(datatarget)
    
    #iterating through dataframe columns ( =  labels)
    for label in list(df.columns):
        
        #create folder
        foldername = datatarget + str(label)
        
        if not path.exists(foldername):
            os.mkdir(foldername)
        
        #move each file
        for file in list(df[label]):
            
            fileoriginal =  datafolder + file + ".jpg"
            filetarget = datatarget +"/" + file + ".jpg"
            
            try:
                if not path.exists(filetarget):
                    shutil.copyfile(fileoriginal, filetarget)

            except FileNotFoundError:
                print("File {} not found!".format(file))
                pass

#origin folder
imagefolder = "../data/food-101/images/"


organise_files_from_df(train_meta, imagefolder, trainfolder)

organise_files_from_df(valid_meta, imagefolder, validfolder)

organise_files_from_df(test_meta, imagefolder, testfolder)

#to delete the origin folder, uncomment the line below
#shutil.rmtree(imagefolder)

## Load Data to S3

>The below cells load in some AWS SageMaker libraries, starts a SageMaker session and creates a default bucket. After creating this bucket, it upload the locally stored data to S3.

In [5]:
import boto3
import sagemaker

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

> ### Upload training and testing data

In [6]:
prefix = "food-classifier"
datafolder = "../data/s3_train_data"
# upload all data to S3

#this is slow!
start = time.time()
input_data = sagemaker_session.upload_data(path=datafolder, bucket=bucket, key_prefix=prefix)
end = time.time()

print("Data uploaded to s3 after {} seconds".format(end - start))

Data uploaded to s3 after 296.4869067668915 seconds


In [10]:
# check that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    #print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('All good!')

All good!


> ## Checking model.py

In [7]:
!pygmentize pytorch_source/model.py

import torch
import torchvision.models as models

#importing pretrained ResNet for transfer learning
ResNetTransfer = models.resnet50(pretrained=True) #.load_state_dict(torch.load("resnet50_base.pt"))


> ## Checking train.py

In [8]:
!pygmentize pytorch_source/train.py

import argparse
import json
import os
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data

from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# imports the model in model.py by name
from model import ResNetTransfer


def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}.".format(device))

    model = ResNetTransfer

    #fre

> ## Create pytorch estimator

In [9]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch, PyTorchModel

In [10]:
# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='pytorch_source', 
                    role=role,
                    framework_version= '1.1.0', #'1.3.1',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'n_classes': nc + 1,  # num of classes for the fully connected layer at the end of the network (defined on the first cells)
                        'n_epochs': 3,
                        'img_short_side_resize':256,
                        'img_input_size':224,
                        'num_workers':16,
                        'batch_size':64
                    })

In [ ]:
estimator.fit({'training': input_data})

> ## Deploy the model for testing

The model will be tested by first deploying it and then sending the testing data to the deployed endpoint

The function that loads the saved model is called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file specified as the entry point.

In [154]:
from sagemaker.predictor import RealTimePredictor

class ImgPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImgPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='application/x-image') #, accept='application/json')

In [155]:
# in case we want to retrieve the model from s3
estimator = PyTorchModel(model_data='s3://sagemaker-eu-central-1-515611759963/food-classifier/sagemaker-pytorch-2020-03-03-15-53-46-306/output/model.tar.gz', 
                             role=role,
                             source_dir='pytorch_source',
                             entry_point='deploy.py',
                            predictor_cls = ImgPredictor,
                           framework_version = '1.1.0')

In [156]:
predictor = estimator.deploy(instance_type='ml.p2.xlarge', initial_instance_count=1)

-------------!

In [157]:
#predictor = sagemaker.predictor.RealTimePredictor(
#    endpoint='sagemaker-pytorch-2020-03-05-13-01-45-046',
#    sagemaker_session = sagemaker_session,
#    content_type='application/x-npy',
#    accept='application/x-npy')

In [158]:
predictor

> ### Transformers set-up for test data

In [149]:
import torch

#Norm values
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

#Img size parameters
img_short_side_resize = 256
img_input_size = 224

import os
from torchvision import datasets
import torchvision.transforms as transforms

transform_test = transforms.Compose([
                    transforms.Resize(img_input_size),  
                    transforms.FiveCrop(img_input_size),
                    transforms.Lambda(lambda crops: torch.stack([transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean = norm_mean, std = norm_std)])(crop) for crop in crops]))])

test_data = datasets.ImageFolder(testfolder, transform_test)

In [150]:
test_data.transforms

StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               FiveCrop(size=(224, 224))
               Lambda()
           )

In [151]:
from PIL import Image
image = Image.open(test_data.imgs[0][0]).convert('RGB')

In [159]:
with open(test_data.imgs[0][0], "rb") as image:
    f = image.read()
    b = bytearray(f)
    print(b[0])

255


In [162]:
a = predictor.predict(b)

In [164]:
type(a)

bytes

> ### Data loaders

In [53]:
import torch

shuffle = True
num_workers = 16
batch_size = 64

test_loader = torch.utils.data.DataLoader(test_data, batch_size=int(np.floor(batch_size/5)), num_workers=0, shuffle=shuffle, pin_memory=True)

> ### Testing function implementation

In [70]:
#TO BE IMPLEMENTED PROPERLY

def test(loader, model, criterion, device):
    """
    test function
    """
    
    test_loss = 0.
    correct = 0.
    total = 0.
    
    #model.eval()
    
    with torch.no_grad():
        
        for batch_idx, (data, target) in enumerate(loader):
            
            data, target = data.to(device), target.to(device) # move to GPU
            
            bs, ncrops, c, h, w = data.size()
            
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model.predict(data.view(-1, c, h, w)) # fuse batch size and ncrops
            output = output.view(bs, ncrops, -1).mean(1)    
            
            loss = criterion(output, target) # calculate the loss
            
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss)) # update average test loss 
            
            pred = output.data.max(1, keepdim=True)[1] # convert output probabilities to predicted class
            
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy()) 
            total += data.size(0)            
            
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

> ## Testing

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
print("Using device {}.".format(device))
if torch.cuda.is_available():
    print("Using",torch.cuda.get_device_name(device))

Using device cpu.


In [72]:
#predictor.predict(test_data)

In [73]:
test(test_loader, predictor, torch.nn.CrossEntropyLoss(), device)

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: tensor([[[[-1.9124, -1.8953, -1.8439,  ..., -2.0152, -2.0494, -2.0665],
          [-1.8953, -1.8610, -1.8610,  ..., -2.0323, -2.0665, -2.0494],
          [-1.8268, -1.8439, -1.8610,  ..., -2.0494, -2.0837, -2.0665],
          ...,
          [ 0.9988,  0.9988,  1.0159,  ...,  1.1358,  1.1529,  1.1700],
          [ 0.9988,  1.0331,  1.0331,  ...,  1.1700,  1.2043,  1.2214],
          [ 0.9817,  0.9988,  1.0502,  ...,  1.1872,  1.2385,  1.2728]],

         [[-1.8431, -1.8256, -1.7731,  ..., -1.9307, -1.9657, -1.9832],
          [-1.7906, -1.7731, -1.7556,  ..., -1.9482, -1.9832, -1.9657],
          [-1.6856, -1.7031, -1.7206,  ..., -1.9657, -2.0007, -1.9832],
          ...,
          [ 1.0805,  1.0980,  1.0980,  ...,  1.2206,  1.2381,  1.2556],
          [ 1.0630,  1.0980,  1.0980,  ...,  1.2556,  1.2906,  1.3081],
          [ 1.0455,  1.0630,  1.1155,  ...,  1.2731,  1.3256,  1.3606]],

         [[

> ## Delete endpoint

In [84]:
estimator.delete_endpoint()

AttributeError: 'PyTorchModel' object has no attribute 'delete_endpoint'